In [2]:
import numpy as np
import pandas as pd
import scipy
from math import log

In [3]:
df = pd.DataFrame(
    [["sales", "senior", 2, 5, 30],
    ["sales", "junior", 1, 1, 40],
    ["sales", "junior", 2, 2, 40],
    ["systems", "junior", 0, 5, 20],
    ["systems", "senior", 2, 6, 5],
    ["systems", "junior", 1, 5, 3],
    ["systems", "senior", 4, 6, 3],
    ["marketing", "senior", 3, 5, 10],
    ["marketing", "junior", 2, 4, 4],
    ["secretary", "senior", 5, 3, 4],
    ["secretary", "junior", 1, 1, 6]],
    columns = ["department", "status", "age", "salary", "count"])

In [4]:
df

,department,status,age,salary,count
0,sales,senior,2,5,30
1,sales,junior,1,1,40
2,sales,junior,2,2,40
3,systems,junior,0,5,20
4,systems,senior,2,6,5
5,systems,junior,1,5,3
6,systems,senior,4,6,3
7,marketing,senior,3,5,10
8,marketing,junior,2,4,4
9,secretary,senior,5,3,4


In [4]:
def gini(df, total):
    #print(df)
    gini = 1
    for row in df.iterrows():
        gini -= (row[1]["count"]/total)**2
    #return 1 - (df["count"][0]/total)**2 - (df["count"][1]/total)**2
    return gini

In [5]:
def split(df, column_name, split):
    df_init = df[["status", "count"]].groupby("status").sum()
    total = df_init["count"].sum()
    # GAIN is calculated by sybsequently calculating each item in sum
    gain = gini(df_init, total)
    split_info = 0
    for group in split:
        df_group = df.loc[df[column_name].isin(group), ["status", "count"]].groupby("status").sum()
        total_group = df_group["count"].sum()
        freq_group = total_group/total
        if freq_group == 0:
            split_info -= 0
        else:
            split_info -= freq_group * log(freq_group)
        gain -= freq_group * gini(df_group, total_group)
    return gain/split_info

In [6]:
def ssq_partition(in_list, k):
    assert len(in_list) >= k
    if k == 1:
        return [[in_list]]
    comb = []
    length = len(in_list)
    for i in range(length - k + 1):
        new_list = in_list[i+1 : length]
        comp_list = [in_list[:i+1]]
        for j in ssq_partition(new_list, k - 1):
            comb.append(comp_list + j)
    return comb

In [7]:
def partition(collection):
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller

In [25]:
age_range = [0, 1, 2, 3, 4, 5]
salary_range = [1, 2, 3, 4, 5, 6]
department_range = ["sales", "systems", "marketing", "secretary"]
splits = {}
splits["department"] = partition(department_range)
splits["age"] = ssq_partition(age_range, 3) + ssq_partition(age_range, 4)
splits["salary"] = ssq_partition(salary_range, 3) + ssq_partition(salary_range, 4)

In [9]:
df_root = pd.DataFrame(columns = ["split", "gainratio"])
gainRatio = 0
index = 0
for column_name, all_col_splits in splits.items():
    for curr_split in all_col_splits:
        #print(curr_split)
        gainRatio = split(df, column_name, curr_split)
        #print(gainRatio)
        df_root.loc[index] = [curr_split, gainRatio]
        index += 1

/Users/gasanoe/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [10]:
df_root.fillna(0, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_root)

                                             split  gainratio
0         [[sales, systems, marketing, secretary]]   0.000000
1       [[sales], [systems, marketing, secretary]]   0.011310
2       [[sales, systems], [marketing, secretary]]   0.059034
3       [[systems], [sales, marketing, secretary]]   0.003121
4     [[sales], [systems], [marketing, secretary]]   0.028383
5       [[sales, systems, marketing], [secretary]]   0.004063
6       [[systems, marketing], [sales, secretary]]   0.009215
7     [[sales], [systems, marketing], [secretary]]   0.009061
8       [[sales, marketing], [systems, secretary]]   0.000595
9       [[marketing], [sales, systems, secretary]]   0.101705
10    [[sales], [marketing], [systems, secretary]]   0.035955
11    [[sales, systems], [marketing], [secretary]]   0.061277
12    [[systems], [sales, marketing], [secretary]]   0.003120
13    [[systems], [marketing], [sales, secretary]]   0.039377
14  [[sales], [systems], [marketing], [secretary]]   0.032721
15      

In [11]:
max(df_root["gainratio"])

0.2819234434432972

In [12]:
df[df["salary"].isin([1, 2])]

,department,status,age,salary,count
1,sales,junior,1,1,40
2,sales,junior,2,2,40
10,secretary,junior,1,1,6


In [13]:
df[df["salary"].isin([3])]

,department,status,age,salary,count
9,secretary,senior,5,3,4


In [15]:
df1 = df[df["salary"].isin([4, 5, 6])]

In [16]:
df1

,department,status,age,salary,count
0,sales,senior,2,5,30
3,systems,junior,0,5,20
4,systems,senior,2,6,5
5,systems,junior,1,5,3
6,systems,senior,4,6,3
7,marketing,senior,3,5,10
8,marketing,junior,2,4,4


In [18]:
df1_root = pd.DataFrame(columns = ["split", "gainratio"])
gainRatio = 0
index = 0
for column_name, all_col_splits in splits.items():
    for curr_split in all_col_splits:
        #print(curr_split)
        gainRatio = split(df1, column_name, curr_split)
        #print(gainRatio)
        df1_root.loc[index] = [curr_split, gainRatio]
        index += 1

/Users/gasanoe/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [19]:
df1_root.fillna(0, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df1_root)

                                             split  gainratio
0         [[sales, systems, marketing, secretary]]   0.000000
1       [[sales], [systems, marketing, secretary]]   0.256756
2       [[sales, systems], [marketing, secretary]]   0.005262
3       [[systems], [sales, marketing, secretary]]   0.303150
4     [[sales], [systems], [marketing, secretary]]   0.216583
5       [[sales, systems, marketing], [secretary]]   0.000000
6       [[systems, marketing], [sales, secretary]]   0.256756
7     [[sales], [systems, marketing], [secretary]]   0.256756
8       [[sales, marketing], [systems, secretary]]   0.303150
9       [[marketing], [sales, systems, secretary]]   0.005262
10    [[sales], [marketing], [systems, secretary]]   0.216583
11    [[sales, systems], [marketing], [secretary]]   0.005262
12    [[systems], [sales, marketing], [secretary]]   0.303150
13    [[systems], [marketing], [sales, secretary]]   0.216583
14  [[sales], [systems], [marketing], [secretary]]   0.216583
15      

In [20]:
max(df1_root["gainratio"])

0.5878228558220879

In [21]:
df1[df1["age"].isin([0, 1])]

,department,status,age,salary,count
3,systems,junior,0,5,20
5,systems,junior,1,5,3


In [24]:
df2 = df1[df1["age"].isin([2, 3, 4])]

In [23]:
df1[df1["age"].isin([5])]

,department,status,age,salary,count


In [26]:
df2_root = pd.DataFrame(columns = ["split", "gainratio"])
gainRatio = 0
index = 0
for column_name, all_col_splits in splits.items():
    for curr_split in all_col_splits:
        #print(curr_split)
        gainRatio = split(df2, column_name, curr_split)
        #print(gainRatio)
        df2_root.loc[index] = [curr_split, gainRatio]
        index += 1

/Users/gasanoe/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [27]:
df2_root.fillna(0, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df2_root)

                                             split  gainratio
0         [[sales, systems, marketing, secretary]]   0.000000
1       [[sales], [systems, marketing, secretary]]   0.023688
2       [[sales, systems], [marketing, secretary]]   0.055145
3       [[systems], [sales, marketing, secretary]]   0.005012
4     [[sales], [systems], [marketing, secretary]]   0.033510
5       [[sales, systems, marketing], [secretary]]   0.000000
6       [[systems, marketing], [sales, secretary]]   0.023688
7     [[sales], [systems, marketing], [secretary]]   0.023688
8       [[sales, marketing], [systems, secretary]]   0.005012
9       [[marketing], [sales, systems, secretary]]   0.055145
10    [[sales], [marketing], [systems, secretary]]   0.033510
11    [[sales, systems], [marketing], [secretary]]   0.055145
12    [[systems], [sales, marketing], [secretary]]   0.005012
13    [[systems], [marketing], [sales, secretary]]   0.033510
14  [[sales], [systems], [marketing], [secretary]]   0.033510
15      

In [30]:
gr_values = sorted(df2_root["gainratio"])
gr_values

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.003284898470003282,
 0.003284898470003282,
 0.003284898470003282,
 0.003284898470003282,
 0.003284898470003282,
 0.003284898470003282,
 0.005011831067690217,
 0.005011831067690217,
 0.005011831067690217,
 0.005011831067690217,
 0.005011831067690217,
 0.005011831067690217,
 0.005011831067690217,
 0.005011831067690217,
 0.005011831067690217,
 0.005656511825801194,
 0.005656511825801194,
 0.005656511825801194,
 0.005656511825801194,
 0.007014986015530497,
 0.007014986015530497,
 0.007014986015530497,
 0.007014986015530497,
 0.007014986015530497,
 0.007014986015530497,
 0.02368783243122856,
 0.02368783243122856,
 0.02368783243122856,
 0.033509523290136095,
 0.033509523290136095,
 0.033509523290136095,
 0.033509523290136095,
 0.055145325567219856,
 0.055145325567219856,
 0.055145325567219856,
 0.20668531821137248,
 0.20668531821137248,
 0.20668531821137248,
 0.20668531821137248,
 0.5236629766374151,
 0.5236629766374151,
 0.52366

In [12]:
df[df["status"]=="junior"]

,department,status,age,salary,count
1,sales,junior,1,1,40
2,sales,junior,2,2,40
3,systems,junior,0,5,20
5,systems,junior,1,5,3
8,marketing,junior,2,4,4
10,secretary,junior,1,1,6


In [7]:
df[df["status"]=="junior"].sum()

department    salessalessystemssystemsmarketingsecretary
status              juniorjuniorjuniorjuniorjuniorjunior
age                                                    7
salary                                                18
count                                                113
dtype: object

In [13]:
df[df["status"]=="senior"]

,department,status,age,salary,count
0,sales,senior,2,5,30
4,systems,senior,2,6,5
6,systems,senior,4,6,3
7,marketing,senior,3,5,10
9,secretary,senior,5,3,4


In [9]:
df[df["status"]=="senior"].sum()

department    salessystemssystemsmarketingsecretary
status               seniorseniorseniorseniorsenior
age                                              16
salary                                           25
count                                            52
dtype: object

In [11]:
df[df["status"]=="junior"][df["department"]=="systems"]

/Users/gasanoe/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,department,status,age,salary,count
3,systems,junior,0,5,20
5,systems,junior,1,5,3
